### В Jupyter: “что у нас вообще есть в БД” (таблицы + колонки + типы)

In [32]:
# db_inspection.ipynb
# Ячейка 1: подключение к PostgreSQL через общий модуль проекта

import sys
from pathlib import Path

# --- 1) Ищем корень проекта: там где лежит папка src/ ---
PROJECT_ROOT = Path.cwd()
while not (PROJECT_ROOT / "src").exists() and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent

if not (PROJECT_ROOT / "src").exists():
    raise RuntimeError("Не найден корень проекта (папка src/). Открой ноутбук внутри проекта.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)

# --- 2) Импортируем общий модуль подключения ---
from src.common.db import get_engine, test_connection

# --- 3) Проверяем подключение ---
test_connection(verbose=True)

# --- 4) Получаем engine для следующих ячеек ---
engine = get_engine()

print("OK: engine created and ready to use")

Project root: /Users/iriska/Downloads/mlbox_2
OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public
OK: engine created and ready to use


In [33]:
import pandas as pd

schema_df = pd.read_sql("select current_schema() as schema;", engine)
SCHEMA = schema_df.loc[0, "schema"]
print("Using schema:", SCHEMA)

Using schema: public


In [34]:
q_tables = """
select table_name, table_type
from information_schema.tables
where table_schema = %s
order by table_type, table_name;
"""
tables = pd.read_sql(q_tables, engine, params=(SCHEMA,))
tables

,table_name,table_type
0,cluster_runs,BASE TABLE
1,context_time_series,BASE TABLE
2,data_sources,BASE TABLE
3,model_runs,BASE TABLE
4,track_cluster_labels,BASE TABLE
5,track_features,BASE TABLE
6,track_points,BASE TABLE
7,track_predictions,BASE TABLE
8,tracks,BASE TABLE
9,v_cluster_distribution_latest,VIEW


In [35]:
q_cols = """
select
  table_name,
  ordinal_position,
  column_name,
  data_type,
  is_nullable,
  column_default
from information_schema.columns
where table_schema = %s
order by table_name, ordinal_position;
"""
cols = pd.read_sql(q_cols, engine, params=(SCHEMA,))
cols

,table_name,ordinal_position,column_name,data_type,is_nullable,column_default
0,cluster_runs,1,cluster_run_id,bigint,NO,nextval('cluster_runs_cluster_run_id_seq'::reg...
1,cluster_runs,2,run_tag,text,NO,None
2,cluster_runs,3,method,text,NO,None
3,cluster_runs,4,created_at,timestamp with time zone,NO,now()
4,cluster_runs,5,silhouette,double precision,YES,None
5,cluster_runs,6,calinski_harabasz,double precision,YES,None
6,cluster_runs,7,davies_bouldin,double precision,YES,None
7,cluster_runs,8,params,jsonb,NO,'{}'::jsonb
8,context_time_series,1,context_id,bigint,NO,nextval('context_time_series_context_id_seq'::...
9,context_time_series,2,source_id,bigint,YES,None


In [39]:
import pandas as pd
from sqlalchemy import text

# engine у тебя уже есть из предыдущей ячейки
# engine = get_engine()

SCHEMA = "public"  # если меняешь схему — меняешь здесь

query = """
SELECT
    table_name,
    table_type
FROM information_schema.tables
WHERE table_schema = :schema
ORDER BY table_type, table_name;
"""

tables = pd.read_sql(
    text(query),
    engine,
    params={"schema": SCHEMA}
)

rows = []

for _, row in tables.iterrows():
    name = row["table_name"]
    ttype = row["table_type"]

    # VIEW считаем отдельно
    if ttype == "VIEW":
        count_sql = f'SELECT COUNT(*) FROM "{SCHEMA}"."{name}"'
    else:
        count_sql = f'SELECT COUNT(*) FROM "{SCHEMA}"."{name}"'

    cnt = pd.read_sql(count_sql, engine).iloc[0, 0]

    rows.append({
        "table": name,
        "type": ttype,
        "rows": cnt
    })

result = pd.DataFrame(rows).sort_values(["type", "table"])
result

,table,type,rows
0,cluster_runs,BASE TABLE,0
1,context_time_series,BASE TABLE,9
2,data_sources,BASE TABLE,56
3,model_runs,BASE TABLE,0
4,track_cluster_labels,BASE TABLE,0
5,track_features,BASE TABLE,42
6,track_points,BASE TABLE,86457
7,track_predictions,BASE TABLE,0
8,tracks,BASE TABLE,42
9,v_cluster_distribution_latest,VIEW,0


In [40]:
from IPython.display import display

# берём только BASE TABLE (без views)
base_tables = tables[tables["table_type"] == "BASE TABLE"]["table_name"].tolist()

for t in base_tables:
    print("\n" + "="*110)
    print(f"TABLE: {t}")
    print("="*110)

    # 1) Показать "шапку" (0 строк) — как ты просил
    df_head = pd.read_sql(f'SELECT * FROM "{SCHEMA}"."{t}" LIMIT 0;', engine)
    display(df_head)

    # 2) Показать первые 10 строк (если есть)
    df_10 = pd.read_sql(f'SELECT * FROM "{SCHEMA}"."{t}" LIMIT 10;', engine)
    if len(df_10) == 0:
        print("(table is empty)")
    else:
        display(df_10)


TABLE: cluster_runs


,cluster_run_id,run_tag,method,created_at,silhouette,calinski_harabasz,davies_bouldin,params


(table is empty)

TABLE: context_time_series


,context_id,source_id,time,lat,lon,values


,context_id,source_id,time,lat,lon,values
0,1,5,2026-02-03 03:22:51+00:00,22.379825,114.176331,"{'point': {'seq': 300, 'ele_m': 3.0, 'segment_..."
1,2,5,2026-02-03 03:27:51+00:00,22.382243,114.172226,"{'point': {'seq': 600, 'ele_m': 36.0, 'segment..."
2,3,5,2026-02-03 03:32:51+00:00,22.382012,114.175528,"{'point': {'seq': 900, 'ele_m': 54.0, 'segment..."
3,4,5,2026-02-03 03:37:51+00:00,22.383420,114.174663,"{'point': {'seq': 1200, 'ele_m': 111.0, 'segme..."
4,5,5,2026-02-03 03:47:51+00:00,22.384778,114.174254,"{'point': {'seq': 1800, 'ele_m': 133.0, 'segme..."
5,6,5,2026-02-03 03:52:51+00:00,22.384401,114.173712,"{'point': {'seq': 2100, 'ele_m': 132.0, 'segme..."
6,7,2,2026-02-01 23:03:48.984000+00:00,53.567858,9.896249,"{'point': {'seq': 1, 'ele_m': 76.0705366265677..."
7,8,3,2026-02-03 00:45:03.999000+00:00,46.188395,-119.217436,"{'point': {'seq': 1, 'ele_m': 234.45009, 'segm..."
8,9,4,2026-01-31 22:07:40+00:00,38.763706,-121.147258,"{'point': {'seq': 1, 'ele_m': 141.800003051757..."


...


TABLE: track_predictions


,model_run_id,track_id,y_pred_num,y_pred_class,y_pred_proba


(table is empty)

TABLE: tracks


,track_id,source_id,ext_track_id,ext_object_id,ext_driver_id,track_name,track_type,start_time,end_time,segment_count,point_count,min_lat,min_lon,max_lat,max_lon,created_at,extra


,track_id,source_id,ext_track_id,ext_object_id,ext_driver_id,track_name,track_type,start_time,end_time,segment_count,point_count,min_lat,min_lon,max_lat,max_lon,created_at,extra
0,2,2,12184656#1,None,None,b2f814774b744fc9_20260202,unknown,2026-02-01 23:03:48.984000+00:00,2026-02-02 22:17:47.734000+00:00,1,45,53.556640,9.875829,53.582463,9.917638,2026-02-03 09:00:01.496846+00:00,"{'has_ele': True, 'has_time': True}"
1,3,3,12184690#1,None,None,12184690,unknown,2026-02-03 00:45:03.999000+00:00,2026-02-03 00:46:02.999000+00:00,1,57,46.188395,-119.218269,46.188579,-119.217436,2026-02-03 09:00:01.503000+00:00,"{'has_ele': True, 'has_time': True}"
2,4,4,12184876#1,None,None,Placer County Trail Running,unknown,2026-01-31 22:07:40+00:00,2026-01-31 22:08:35+00:00,1,56,38.763658,-121.148898,38.763794,-121.147258,2026-02-03 09:00:01.507930+00:00,"{'has_ele': True, 'has_time': True}"
3,5,5,12184877#1,None,None,Lunch Trail Run,unknown,2026-02-03 03:17:52+00:00,2026-02-03 04:24:02+00:00,1,3971,22.374993,114.172217,22.388597,114.187769,2026-02-03 09:00:01.514405+00:00,"{'has_ele': True, 'has_time': True}"
4,6,6,Alt_Portsmouth#1,None,None,127 A27 - 21 Wharf Rd,unknown,NaT,NaT,1,1219,50.812740,-1.282590,50.875670,-1.074060,2026-02-03 09:00:01.759386+00:00,"{'has_ele': True, 'has_time': False}"
5,7,7,Alt_Portsmouth.gpx_0#1,None,None,Alt_Portsmouth.gpx_0,unknown,NaT,NaT,1,180,50.812740,-1.282590,50.875670,-1.074070,2026-02-03 09:00:01.808081+00:00,"{'has_ele': False, 'has_time': False}"
6,8,8,Basel_St-Brevin_Eurovelo6#1,None,None,EV6_Bale_StBrevin,unknown,2010-01-01 00:43:23+00:00,2012-09-02 08:17:40+00:00,1,21240,46.442299,-2.167251,47.901333,7.592952,2026-02-03 09:00:01.821020+00:00,"{'has_ele': True, 'has_time': True}"
7,9,9,Basel_St-Brevin_Eurovelo6.gpx_0#1,None,None,Basel_St-Brevin_Eurovelo6.gpx_0,unknown,NaT,NaT,1,8049,46.442300,-2.167250,47.901330,7.592950,2026-02-03 09:00:02.623146+00:00,"{'has_ele': False, 'has_time': False}"
8,10,10,Basel_St-Brevin_Eurovelo6_1#1,None,None,EV6_Bale_StBrevin section 1,unknown,NaT,NaT,1,1459,47.078560,5.476800,47.768540,7.592950,2026-02-03 09:00:02.895790+00:00,"{'has_ele': False, 'has_time': False}"
9,11,11,Basel_St-Brevin_Eurovelo6_2#1,None,None,EV6_Bale_StBrevin section 2,unknown,NaT,NaT,1,1706,46.442300,3.741490,47.107860,5.476800,2026-02-03 09:00:02.949431+00:00,"{'has_ele': False, 'has_time': False}"


In [13]:
cols[(cols.table_name=="track_points") & (cols.column_name=="extra")]

,table_name,ordinal_position,column_name,data_type,is_nullable,column_default
57,track_points,13,extra,jsonb,NO,'{}'::jsonb


## Очистка базы

In [7]:
from sqlalchemy import text

# SCHEMA у тебя уже определён (current_schema()).
wipe_sql = f"""
TRUNCATE TABLE
  "{SCHEMA}"."track_points",
  "{SCHEMA}"."track_features",
  "{SCHEMA}"."track_cluster_labels",
  "{SCHEMA}"."track_predictions",
  "{SCHEMA}"."tracks",
  "{SCHEMA}"."data_sources",
  "{SCHEMA}"."cluster_runs",
  "{SCHEMA}"."model_runs",
  "{SCHEMA}"."context_time_series"
RESTART IDENTITY CASCADE;
"""

with engine.begin() as conn:
    conn.execute(text(wipe_sql))

print("OK: all data wiped; schema kept")

OK: all data wiped; schema kept


## Выполнить после развертывания БД

In [9]:
from sqlalchemy import text

# ВАЖНО:
# SCHEMA лучше брать так, как ты уже делал: current_schema()
# но если у тебя уже есть переменная SCHEMA — оставь как есть.
# Пример:
# SCHEMA = pd.read_sql("select current_schema() as schema;", engine).loc[0, "schema"]

sql = """
-- =========================================================
-- 1) DEFAULT '{{}}' для jsonb (чтобы NULL не валил вставки)
-- =========================================================

ALTER TABLE "{schema}"."data_sources"
  ALTER COLUMN extra SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."tracks"
  ALTER COLUMN extra SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."track_points"
  ALTER COLUMN extra SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."track_features"
  ALTER COLUMN extra SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."cluster_runs"
  ALTER COLUMN params SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."model_runs"
  ALTER COLUMN params SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."model_runs"
  ALTER COLUMN metrics SET DEFAULT '{{}}'::jsonb;

ALTER TABLE "{schema}"."context_time_series"
  ALTER COLUMN values SET DEFAULT '{{}}'::jsonb;


-- =========================================================
-- 2) UNIQUE (защита от дублей при повторных запусках)
-- =========================================================

-- 2.1) Один файл = один hash (иначе при повторе будет дублирование)
DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conname = 'uq_data_sources_file_hash'
      AND conrelid = '"{schema}"."data_sources"'::regclass
  ) THEN
    ALTER TABLE "{schema}"."data_sources"
      ADD CONSTRAINT uq_data_sources_file_hash UNIQUE (file_hash);
  END IF;
END $$;

-- 2.2) Уникальность точки внутри трека (seq + segment_index)
DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conname = 'uq_track_points_track_seq_seg'
      AND conrelid = '"{schema}"."track_points"'::regclass
  ) THEN
    ALTER TABLE "{schema}"."track_points"
      ADD CONSTRAINT uq_track_points_track_seq_seg UNIQUE (track_id, seq, segment_index);
  END IF;
END $$;


-- =========================================================
-- 3) Индексы под скорость дашборда и аналитики
-- =========================================================

CREATE INDEX IF NOT EXISTS ix_tracks_source_id
  ON "{schema}"."tracks"(source_id);

CREATE INDEX IF NOT EXISTS ix_tracks_start_time
  ON "{schema}"."tracks"(start_time);

CREATE INDEX IF NOT EXISTS ix_track_points_track_id
  ON "{schema}"."track_points"(track_id);

CREATE INDEX IF NOT EXISTS ix_track_points_time
  ON "{schema}"."track_points"(time);

CREATE INDEX IF NOT EXISTS ix_track_points_lat_lon
  ON "{schema}"."track_points"(lat, lon);
""".format(schema=SCHEMA)

with engine.begin() as conn:
    conn.execute(text(sql))

print("OK: DB hardening applied (defaults / unique / indexes)")

OK: DB hardening applied (defaults / unique / indexes)


In [19]:
import pandas as pd
from sqlalchemy import text

# engine у тебя уже есть из предыдущей ячейки
# engine = get_engine()

print("="*80)
print("1) ПРОВЕРКА КООРДИНАТ (lat/lon)")
print("="*80)

q_lat0 = """
SELECT COUNT(*) AS lat_zero
FROM public.track_points
WHERE lat = 0;
"""
print("lat = 0:", pd.read_sql(q_lat0, engine).iloc[0,0])

q_bad_lat = """
SELECT COUNT(*) AS lat_bad
FROM public.track_points
WHERE lat IS NOT NULL AND (lat < -90 OR lat > 90);
"""
print("lat вне диапазона [-90,90]:", pd.read_sql(q_bad_lat, engine).iloc[0,0])

q_bad_lon = """
SELECT COUNT(*) AS lon_bad
FROM public.track_points
WHERE lon IS NOT NULL AND (lon < -180 OR lon > 180);
"""
print("lon вне диапазона [-180,180]:", pd.read_sql(q_bad_lon, engine).iloc[0,0])


print("\n" + "="*80)
print("2) ПРОВЕРКА ВРЕМЕНИ")
print("="*80)

q_time = """
SELECT
    COUNT(*) FILTER (WHERE time IS NOT NULL) AS with_time,
    COUNT(*) FILTER (WHERE time IS NULL) AS without_time
FROM public.track_points;
"""
print(pd.read_sql(q_time, engine))


print("\n" + "="*80)
print("3) ПРИМЕРЫ ТОЧЕК (первые 10 строк)")
print("="*80)

q_sample = """
SELECT
    track_id,
    segment_index,
    seq,
    lat,
    lon,
    ele,
    time,
    speed
FROM public.track_points
ORDER BY track_id, segment_index, seq
LIMIT 10;
"""
display(pd.read_sql(q_sample, engine))


print("\n" + "="*80)
print("4) СВОДКА ПО ТРЕКАМ")
print("="*80)

q_tracks = """
SELECT
    COUNT(*) AS tracks_total,
    COUNT(*) FILTER (WHERE start_time IS NOT NULL) AS tracks_with_time,
    COUNT(*) FILTER (WHERE start_time IS NULL) AS tracks_without_time,
    MIN(point_count) AS min_points,
    MAX(point_count) AS max_points
FROM public.tracks;
"""
display(pd.read_sql(q_tracks, engine))


print("\n" + "="*80)
print("5) ПРОВЕРКА СООТВЕТСТВИЯ POINTS ↔ TRACKS")
print("="*80)

q_join = """
SELECT
    COUNT(DISTINCT t.track_id) AS tracks,
    COUNT(p.point_id) AS points
FROM public.tracks t
LEFT JOIN public.track_points p ON p.track_id = t.track_id;
"""
display(pd.read_sql(q_join, engine))

print("\n✅ ПРОВЕРКА ЗАВЕРШЕНА")

1) ПРОВЕРКА КООРДИНАТ (lat/lon)
lat = 0: 0
lat вне диапазона [-90,90]: 0
lon вне диапазона [-180,180]: 0

2) ПРОВЕРКА ВРЕМЕНИ
   with_time  without_time
0      11889         74568

3) ПРИМЕРЫ ТОЧЕК (первые 10 строк)


,track_id,segment_index,seq,lat,lon,ele,time,speed
0,2,0,1,53.567858,9.896249,76.070537,2026-02-01 23:03:48.984000+00:00,None
1,2,0,2,53.567639,9.896037,16.976774,2026-02-02 01:41:51.197000+00:00,None
2,2,0,3,53.567774,9.896244,38.920773,2026-02-02 01:43:53.882000+00:00,None
3,2,0,4,53.568025,9.896087,71.289448,2026-02-02 01:50:02.781000+00:00,None
4,2,0,5,53.567885,9.896283,78.701094,2026-02-02 01:52:06.155000+00:00,None
5,2,0,6,53.567699,9.896212,56.790838,2026-02-02 01:54:12.503000+00:00,None
6,2,0,7,53.567927,9.896153,100.796573,2026-02-02 01:56:14.952000+00:00,None
7,2,0,8,53.567703,9.896153,71.473557,2026-02-02 02:24:56.933000+00:00,None
8,2,0,9,53.567862,9.896332,85.068081,2026-02-02 02:26:59.445000+00:00,None
9,2,0,10,53.567859,9.896022,101.727065,2026-02-02 03:26:17.426000+00:00,None



4) СВОДКА ПО ТРЕКАМ


,tracks_total,tracks_with_time,tracks_without_time,min_points,max_points
0,42,9,33,45,21240



5) ПРОВЕРКА СООТВЕТСТВИЯ POINTS ↔ TRACKS


,tracks,points
0,42,86457



✅ ПРОВЕРКА ЗАВЕРШЕНА


In [30]:
import pandas as pd
from sqlalchemy import text

# engine уже должен быть создан через get_engine()
# from src.common.db import get_engine
# engine = get_engine()

print("="*90)
print("CHECK 1) ОБЪЁМ ДАННЫХ В ОСНОВНЫХ ТАБЛИЦАХ")
print("="*90)

q_counts = """
SELECT 'data_sources' AS table, COUNT(*) AS rows FROM data_sources
UNION ALL
SELECT 'tracks', COUNT(*) FROM tracks
UNION ALL
SELECT 'track_points', COUNT(*) FROM track_points
UNION ALL
SELECT 'track_features', COUNT(*) FROM track_features;
"""
display(pd.read_sql(q_counts, engine))


print("\n" + "="*90)
print("CHECK 2) СООТВЕТСТВИЕ tracks ↔ track_points")
print("="*90)

q_tp = """
SELECT
  COUNT(DISTINCT t.track_id) AS tracks,
  COUNT(tp.point_id) AS points
FROM tracks t
LEFT JOIN track_points tp ON tp.track_id = t.track_id;
"""
display(pd.read_sql(q_tp, engine))


print("\n" + "="*90)
print("CHECK 3) КООРДИНАТЫ (lat/lon)")
print("="*90)

q_coords = """
SELECT
  SUM(CASE WHEN lat = 0 THEN 1 ELSE 0 END) AS lat_zero,
  SUM(CASE WHEN lat < -90 OR lat > 90 THEN 1 ELSE 0 END) AS lat_out,
  SUM(CASE WHEN lon < -180 OR lon > 180 THEN 1 ELSE 0 END) AS lon_out
FROM track_points;
"""
display(pd.read_sql(q_coords, engine))


print("\n" + "="*90)
print("CHECK 4) ВРЕМЯ В ТОЧКАХ")
print("="*90)

q_time = """
SELECT
  SUM(CASE WHEN time IS NOT NULL THEN 1 ELSE 0 END) AS with_time,
  SUM(CASE WHEN time IS NULL THEN 1 ELSE 0 END) AS without_time
FROM track_points;
"""
display(pd.read_sql(q_time, engine))


print("\n" + "="*90)
print("CHECK 5) АНОМАЛИИ В track_features")
print("="*90)

q_bad = """
SELECT
  track_id,
  distance_m,
  duration_s,
  avg_speed_mps,
  stop_time_s,
  stop_ratio
FROM track_features
WHERE
  (duration_s IS NOT NULL AND duration_s > 7*24*3600)    -- > 7 дней
  OR (stop_time_s IS NOT NULL AND stop_time_s < 0)
  OR (stop_ratio IS NOT NULL AND (stop_ratio < 0 OR stop_ratio > 1))
ORDER BY track_id;
"""
bad = pd.read_sql(q_bad, engine)

if bad.empty:
    print("OK ✅ аномалий не найдено")
else:
    print("WARN ⚠️ найдены подозрительные треки:")
    display(bad)


print("\n" + "="*90)
print("CHECK 6) ПОКРЫТИЕ ФИЧЕЙ")
print("="*90)

q_coverage = """
SELECT
  COUNT(*) AS tracks_total,
  SUM(CASE WHEN duration_s IS NOT NULL THEN 1 ELSE 0 END) AS with_duration,
  SUM(CASE WHEN elev_min_m IS NOT NULL THEN 1 ELSE 0 END) AS with_elevation,
  SUM(CASE WHEN avg_speed_mps IS NOT NULL THEN 1 ELSE 0 END) AS with_speed
FROM track_features;
"""
display(pd.read_sql(q_coverage, engine))


print("\n" + "="*90)
print("CHECK 7) ПРИМЕРЫ ФИЧЕЙ (первые 10)")
print("="*90)

display(pd.read_sql("SELECT * FROM track_features ORDER BY track_id LIMIT 10;", engine))

print("\n✅ ВСЕ ПРОВЕРКИ ЗАВЕРШЕНЫ")

CHECK 1) ОБЪЁМ ДАННЫХ В ОСНОВНЫХ ТАБЛИЦАХ


,table,rows
0,data_sources,56
1,tracks,42
2,track_points,86457
3,track_features,42



CHECK 2) СООТВЕТСТВИЕ tracks ↔ track_points


,tracks,points
0,42,86457



CHECK 3) КООРДИНАТЫ (lat/lon)


...


CHECK 6) ПОКРЫТИЕ ФИЧЕЙ


,tracks_total,with_duration,with_elevation,with_speed
0,42,9,19,9



CHECK 7) ПРИМЕРЫ ФИЧЕЙ (первые 10)


,track_id,distance_m,duration_s,elev_min_m,elev_max_m,elev_gain_m,elev_loss_m,avg_speed_mps,max_speed_mps,stop_time_s,stop_ratio,point_density_per_km,updated_at,extra
0,2,1.114268e+04,83638.75,16.976774,130.288880,464.808437,448.883318,0.133224,None,0.000000e+00,0.000000e+00,4.038526,2026-02-03 09:34:19.632632+00:00,"{'points': 45, 'has_ele': True, 'has_time': Tr..."
1,3,1.034899e+02,59.00,233.516740,237.162220,5.893180,3.181050,1.754065,None,0.000000e+00,0.000000e+00,550.778611,2026-02-03 09:34:19.642097+00:00,"{'points': 57, 'has_ele': True, 'has_time': Tr..."
2,4,1.473631e+02,55.00,137.800003,141.800003,0.399994,4.399994,2.679330,None,0.000000e+00,0.000000e+00,380.013659,2026-02-03 09:34:19.646181+00:00,"{'points': 56, 'has_ele': True, 'has_time': Tr..."
3,5,5.998970e+03,3970.00,-1.000000,296.000000,422.000000,411.000000,1.511075,None,1.622000e+03,4.085642e-01,661.947000,2026-02-03 09:34:19.663222+00:00,"{'points': 3971, 'has_ele': True, 'has_time': ..."
4,6,2.173584e+04,NaN,-1.000000,56.000000,204.842000,242.342000,NaN,None,NaN,NaN,56.082479,2026-02-03 09:34:19.685171+00:00,"{'points': 1219, 'has_ele': True, 'has_time': ..."
5,7,2.154706e+04,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,8.353809,2026-02-03 09:34:19.690553+00:00,"{'points': 180, 'has_ele': False, 'has_time': ..."
6,8,1.209975e+06,84265352.00,-6.960000,375.265625,5878.691272,6119.542834,0.014359,None,-3.421871e+21,-4.060828e+13,17.554087,2026-02-03 09:34:19.739079+00:00,"{'points': 21240, 'has_ele': True, 'has_time':..."
7,9,1.207498e+06,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,6.665849,2026-02-03 09:34:19.755572+00:00,"{'points': 8049, 'has_ele': False, 'has_time':..."
8,10,2.415557e+05,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,6.040014,2026-02-03 09:34:19.761725+00:00,"{'points': 1459, 'has_ele': False, 'has_time':..."
9,11,2.421322e+05,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,7.045737,2026-02-03 09:34:19.767601+00:00,"{'points': 1706, 'has_ele': False, 'has_time':..."



✅ ВСЕ ПРОВЕРКИ ЗАВЕРШЕНЫ


In [21]:
# ============================================================
# GPX PARSING CHECK (tracks + track_points + data_sources)
# Универсальная проверка после src.ingest_gpx
# ============================================================

import sys
from pathlib import Path
import pandas as pd
from sqlalchemy import text

# --- 1) Подключаемся через общий модуль проекта (без паролей в ноутбуке) ---
PROJECT_ROOT = Path.cwd()
# если ноутбук лежит в data/ или sql/ — поднимемся на уровень выше
if (PROJECT_ROOT / "src").exists() is False:
    PROJECT_ROOT = PROJECT_ROOT.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.common.db import get_engine, test_connection

test_connection(verbose=True)
engine = get_engine()

# --- 2) Определяем схему (если в твоём проекте cfg.schema есть — возьми оттуда) ---
# ВАЖНО: в твоих текущих скриптах схема берётся из .env/конфига.
# Здесь для проверки используем current_schema(), чтобы не хардкодить public.
with engine.begin() as conn:
    SCHEMA = conn.execute(text("select current_schema()")).scalar()

print(f"\nUsing schema: {SCHEMA}")

# --- Вспомогательная функция: безопасно взять число строк ---
def get_count(conn, schema: str, name: str) -> int:
    return conn.execute(text(f'SELECT COUNT(*) FROM "{schema}"."{name}"')).scalar()

# ============================================================
# CHECK 1) Объёмы основных таблиц
# ============================================================
with engine.begin() as conn:
    counts = {
        "data_sources": get_count(conn, SCHEMA, "data_sources"),
        "tracks": get_count(conn, SCHEMA, "tracks"),
        "track_points": get_count(conn, SCHEMA, "track_points"),
        "track_features": None,
    }
    # track_features может быть не заполнена — но таблица есть
    try:
        counts["track_features"] = get_count(conn, SCHEMA, "track_features")
    except Exception:
        counts["track_features"] = "(table missing?)"

df_counts = pd.DataFrame([{"table": k, "rows": v} for k, v in counts.items()])
print("\n" + "="*90)
print("CHECK 1) ОБЪЁМ ДАННЫХ В ОСНОВНЫХ ТАБЛИЦАХ")
print("="*90)
display(df_counts)

# ============================================================
# CHECK 2) Связность tracks ↔ track_points (каждая точка должна иметь track_id)
# ============================================================
q_links = f"""
SELECT
  (SELECT COUNT(*) FROM "{SCHEMA}"."tracks") AS tracks,
  (SELECT COUNT(*) FROM "{SCHEMA}"."track_points") AS points,
  (SELECT COUNT(*) FROM "{SCHEMA}"."track_points" p
      LEFT JOIN "{SCHEMA}"."tracks" t ON t.track_id = p.track_id
      WHERE t.track_id IS NULL) AS points_without_track
"""
df_links = pd.read_sql(text(q_links), engine)

print("\n" + "="*90)
print("CHECK 2) СООТВЕТСТВИЕ tracks ↔ track_points")
print("="*90)
display(df_links)

# ============================================================
# CHECK 3) Координаты (lat/lon) — диапазоны и нули
# ============================================================
q_coords = f"""
SELECT
  SUM(CASE WHEN lat IS NULL OR lon IS NULL THEN 1 ELSE 0 END) AS null_latlon,
  SUM(CASE WHEN lat = 0 OR lon = 0 THEN 1 ELSE 0 END) AS zero_latlon,
  SUM(CASE WHEN lat < -90 OR lat > 90 THEN 1 ELSE 0 END) AS lat_out,
  SUM(CASE WHEN lon < -180 OR lon > 180 THEN 1 ELSE 0 END) AS lon_out
FROM "{SCHEMA}"."track_points"
"""
df_coords = pd.read_sql(text(q_coords), engine)

print("\n" + "="*90)
print("CHECK 3) КООРДИНАТЫ (lat/lon)")
print("="*90)
display(df_coords)

# ============================================================
# CHECK 4) Время: сколько точек с time / без time, и насколько оно адекватно
# ============================================================
q_time = f"""
SELECT
  SUM(CASE WHEN time IS NOT NULL THEN 1 ELSE 0 END) AS with_time,
  SUM(CASE WHEN time IS NULL THEN 1 ELSE 0 END) AS without_time
FROM "{SCHEMA}"."track_points"
"""
df_time = pd.read_sql(text(q_time), engine)

print("\n" + "="*90)
print("CHECK 4) ВРЕМЯ В ТОЧКАХ")
print("="*90)
display(df_time)

# Проверка: start_time <= end_time в tracks (если оба есть)
q_track_time_order = f"""
SELECT COUNT(*) AS bad_tracks_time_order
FROM "{SCHEMA}"."tracks"
WHERE start_time IS NOT NULL AND end_time IS NOT NULL AND end_time < start_time
"""
df_time_order = pd.read_sql(text(q_track_time_order), engine)

print("\n" + "="*90)
print("CHECK 4.1) ПОРЯДОК ВРЕМЕНИ В TRACKS (end_time не должен быть раньше start_time)")
print("="*90)
display(df_time_order)

# ============================================================
# CHECK 5) Дубли точек по (track_id, seq) — это частая проблема
# ============================================================
q_dups = f"""
SELECT track_id, seq, COUNT(*) AS cnt
FROM "{SCHEMA}"."track_points"
GROUP BY track_id, seq
HAVING COUNT(*) > 1
ORDER BY cnt DESC
LIMIT 20
"""
df_dups = pd.read_sql(text(q_dups), engine)

print("\n" + "="*90)
print("CHECK 5) ДУБЛИ ТОЧЕК ПО (track_id, seq) — если пусто, значит ок")
print("="*90)
display(df_dups)

# ============================================================
# CHECK 6) Пустые / подозрительные треки
#   - point_count должен совпадать с реальным количеством точек
#   - min/max lat/lon должны быть не NULL
# ============================================================
q_tracks_integrity = f"""
WITH p AS (
  SELECT track_id, COUNT(*) AS real_points
  FROM "{SCHEMA}"."track_points"
  GROUP BY track_id
)
SELECT
  t.track_id,
  t.point_count,
  p.real_points,
  t.min_lat, t.min_lon, t.max_lat, t.max_lon,
  t.start_time, t.end_time
FROM "{SCHEMA}"."tracks" t
LEFT JOIN p ON p.track_id = t.track_id
WHERE
  p.real_points IS NULL
  OR t.point_count IS NULL
  OR p.real_points <> t.point_count
  OR t.min_lat IS NULL OR t.min_lon IS NULL OR t.max_lat IS NULL OR t.max_lon IS NULL
ORDER BY t.track_id
LIMIT 50
"""
df_bad_tracks = pd.read_sql(text(q_tracks_integrity), engine)

print("\n" + "="*90)
print("CHECK 6) ПОДОЗРИТЕЛЬНЫЕ TRACKS (несовпадение point_count или NULL bbox)")
print("="*90)
display(df_bad_tracks)

# ============================================================
# CHECK 7) Примеры данных (чтобы глазами увидеть “как в таблице”)
# ============================================================
q_sample_tracks = f"""
SELECT track_id, source_id, track_name, start_time, end_time, segment_count, point_count,
       min_lat, min_lon, max_lat, max_lon, created_at
FROM "{SCHEMA}"."tracks"
ORDER BY track_id
LIMIT 10
"""
q_sample_points = f"""
SELECT track_id, segment_index, seq, lat, lon, ele, time, speed
FROM "{SCHEMA}"."track_points"
ORDER BY track_id, seq
LIMIT 10
"""
df_sample_tracks = pd.read_sql(text(q_sample_tracks), engine)
df_sample_points = pd.read_sql(text(q_sample_points), engine)

print("\n" + "="*90)
print("CHECK 7) ПРИМЕРЫ TRACKS (первые 10)")
print("="*90)
display(df_sample_tracks)

print("\n" + "="*90)
print("CHECK 7.1) ПРИМЕРЫ TRACK_POINTS (первые 10)")
print("="*90)
display(df_sample_points)

print("\n✅ ПРОВЕРКА ПАРСИНГА ЗАВЕРШЕНА")

OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public

Using schema: public

CHECK 1) ОБЪЁМ ДАННЫХ В ОСНОВНЫХ ТАБЛИЦАХ


,table,rows
0,data_sources,56
1,tracks,42
2,track_points,86457
3,track_features,42



CHECK 2) СООТВЕТСТВИЕ tracks ↔ track_points


,tracks,points,points_without_track
0,42,86457,0



CHECK 3) КООРДИНАТЫ (lat/lon)


...


CHECK 7) ПРИМЕРЫ TRACKS (первые 10)


,track_id,source_id,track_name,start_time,end_time,segment_count,point_count,min_lat,min_lon,max_lat,max_lon,created_at
0,2,2,b2f814774b744fc9_20260202,2026-02-01 23:03:48.984000+00:00,2026-02-02 22:17:47.734000+00:00,1,45,53.556640,9.875829,53.582463,9.917638,2026-02-03 09:00:01.496846+00:00
1,3,3,12184690,2026-02-03 00:45:03.999000+00:00,2026-02-03 00:46:02.999000+00:00,1,57,46.188395,-119.218269,46.188579,-119.217436,2026-02-03 09:00:01.503000+00:00
2,4,4,Placer County Trail Running,2026-01-31 22:07:40+00:00,2026-01-31 22:08:35+00:00,1,56,38.763658,-121.148898,38.763794,-121.147258,2026-02-03 09:00:01.507930+00:00
3,5,5,Lunch Trail Run,2026-02-03 03:17:52+00:00,2026-02-03 04:24:02+00:00,1,3971,22.374993,114.172217,22.388597,114.187769,2026-02-03 09:00:01.514405+00:00
4,6,6,127 A27 - 21 Wharf Rd,NaT,NaT,1,1219,50.812740,-1.282590,50.875670,-1.074060,2026-02-03 09:00:01.759386+00:00
5,7,7,Alt_Portsmouth.gpx_0,NaT,NaT,1,180,50.812740,-1.282590,50.875670,-1.074070,2026-02-03 09:00:01.808081+00:00
6,8,8,EV6_Bale_StBrevin,2010-01-01 00:43:23+00:00,2012-09-02 08:17:40+00:00,1,21240,46.442299,-2.167251,47.901333,7.592952,2026-02-03 09:00:01.821020+00:00
7,9,9,Basel_St-Brevin_Eurovelo6.gpx_0,NaT,NaT,1,8049,46.442300,-2.167250,47.901330,7.592950,2026-02-03 09:00:02.623146+00:00
8,10,10,EV6_Bale_StBrevin section 1,NaT,NaT,1,1459,47.078560,5.476800,47.768540,7.592950,2026-02-03 09:00:02.895790+00:00
9,11,11,EV6_Bale_StBrevin section 2,NaT,NaT,1,1706,46.442300,3.741490,47.107860,5.476800,2026-02-03 09:00:02.949431+00:00



CHECK 7.1) ПРИМЕРЫ TRACK_POINTS (первые 10)


,track_id,segment_index,seq,lat,lon,ele,time,speed
0,2,0,1,53.567858,9.896249,76.070537,2026-02-01 23:03:48.984000+00:00,None
1,2,0,2,53.567639,9.896037,16.976774,2026-02-02 01:41:51.197000+00:00,None
2,2,0,3,53.567774,9.896244,38.920773,2026-02-02 01:43:53.882000+00:00,None
3,2,0,4,53.568025,9.896087,71.289448,2026-02-02 01:50:02.781000+00:00,None
4,2,0,5,53.567885,9.896283,78.701094,2026-02-02 01:52:06.155000+00:00,None
5,2,0,6,53.567699,9.896212,56.790838,2026-02-02 01:54:12.503000+00:00,None
6,2,0,7,53.567927,9.896153,100.796573,2026-02-02 01:56:14.952000+00:00,None
7,2,0,8,53.567703,9.896153,71.473557,2026-02-02 02:24:56.933000+00:00,None
8,2,0,9,53.567862,9.896332,85.068081,2026-02-02 02:26:59.445000+00:00,None
9,2,0,10,53.567859,9.896022,101.727065,2026-02-02 03:26:17.426000+00:00,None



✅ ПРОВЕРКА ПАРСИНГА ЗАВЕРШЕНА


In [23]:
# CHECK: track_features quality & consistency (FIXED: no reserved words)
import pandas as pd
from sqlalchemy import text

from src.common.db import get_engine, test_connection

test_connection(verbose=True)
engine = get_engine()

def show(title: str):
    print("\n" + "="*90)
    print(title)
    print("="*90)

with engine.begin() as conn:
    # 0) объемы и связность
    show("0) ОБЪЁМ И СВЯЗНОСТЬ tracks / track_points / track_features")
    q_counts = """
    SELECT 'tracks'::text AS tbl, COUNT(*)::bigint AS rows FROM tracks
    UNION ALL
    SELECT 'track_points'::text AS tbl, COUNT(*)::bigint AS rows FROM track_points
    UNION ALL
    SELECT 'track_features'::text AS tbl, COUNT(*)::bigint AS rows FROM track_features
    ORDER BY tbl;
    """
    print(pd.read_sql(text(q_counts), conn))

    q_missing_features = """
    SELECT COUNT(*)::bigint AS tracks_without_features
    FROM tracks t
    LEFT JOIN track_features f ON f.track_id = t.track_id
    WHERE f.track_id IS NULL;
    """
    print(pd.read_sql(text(q_missing_features), conn))

    q_missing_tracks = """
    SELECT COUNT(*)::bigint AS features_without_tracks
    FROM track_features f
    LEFT JOIN tracks t ON t.track_id = f.track_id
    WHERE t.track_id IS NULL;
    """
    print(pd.read_sql(text(q_missing_tracks), conn))

    # 1) базовые диапазоны / NULL покрытия
    show("1) ПОКРЫТИЕ И NULL-ПРОФИЛЬ ФИЧЕЙ (сколько треков с данными)")
    q_coverage = """
    SELECT
      COUNT(*)::int AS tracks_total,
      SUM((distance_m IS NOT NULL)::int) AS with_distance,
      SUM((duration_s IS NOT NULL AND duration_s > 0)::int) AS with_duration,
      SUM((avg_speed_mps IS NOT NULL)::int) AS with_avg_speed,
      SUM((max_speed_mps IS NOT NULL)::int) AS with_max_speed,
      SUM((elev_min_m IS NOT NULL)::int) AS with_elev_min,
      SUM((elev_gain_m IS NOT NULL)::int) AS with_elev_gain,
      SUM((stop_time_s IS NOT NULL)::int) AS with_stop_time,
      SUM((stop_ratio IS NOT NULL)::int) AS with_stop_ratio
    FROM track_features;
    """
    print(pd.read_sql(text(q_coverage), conn))

    # 2) согласованность distance/duration/speed
    show("2) СОГЛАСОВАННОСТЬ distance ↔ duration ↔ avg_speed")
    q_speed_consistency = """
    SELECT
      track_id,
      distance_m,
      duration_s,
      avg_speed_mps,
      CASE
        WHEN duration_s IS NULL OR duration_s <= 0 OR distance_m IS NULL THEN NULL
        ELSE distance_m / duration_s
      END AS speed_calc_mps,
      CASE
        WHEN duration_s IS NULL OR duration_s <= 0 OR avg_speed_mps IS NULL OR distance_m IS NULL THEN NULL
        ELSE ABS(avg_speed_mps - (distance_m / duration_s))
      END AS abs_diff
    FROM track_features
    WHERE duration_s IS NOT NULL AND duration_s > 0
      AND avg_speed_mps IS NOT NULL
      AND distance_m IS NOT NULL
    ORDER BY abs_diff DESC NULLS LAST
    LIMIT 20;
    """
    print(pd.read_sql(text(q_speed_consistency), conn))

    # 3) stop_time/stop_ratio sanity
    show("3) ПРОВЕРКА stop_time_s / stop_ratio АНОМАЛИИ")
    q_stop_anoms = """
    SELECT track_id, duration_s, stop_time_s, stop_ratio
    FROM track_features
    WHERE
      (stop_time_s IS NOT NULL AND stop_time_s < 0)
      OR (duration_s IS NOT NULL AND stop_time_s IS NOT NULL AND stop_time_s > duration_s * 1.05)
      OR (stop_ratio IS NOT NULL AND (stop_ratio < 0 OR stop_ratio > 1.05))
    ORDER BY
      (stop_time_s < 0) DESC,
      (stop_ratio < 0 OR stop_ratio > 1.05) DESC,
      stop_time_s DESC NULLS LAST
    LIMIT 50;
    """
    df_stop = pd.read_sql(text(q_stop_anoms), conn)
    if df_stop.empty:
        print("OK ✅ аномалий stop_time/stop_ratio не найдено")
    else:
        print("WARN ⚠️ найдены аномалии stop_time/stop_ratio:")
        print(df_stop)

    # 4) Elevation sanity
    show("4) ПРОВЕРКА ВЫСОТ (elev_*)")
    q_ele_anoms = """
    SELECT track_id, elev_min_m, elev_max_m, elev_gain_m, elev_loss_m
    FROM track_features
    WHERE
      (elev_min_m IS NOT NULL AND elev_max_m IS NOT NULL AND elev_min_m > elev_max_m)
      OR (elev_gain_m IS NOT NULL AND elev_gain_m < 0)
      OR (elev_loss_m IS NOT NULL AND elev_loss_m < 0)
    ORDER BY track_id
    LIMIT 50;
    """
    df_ele = pd.read_sql(text(q_ele_anoms), conn)
    if df_ele.empty:
        print("OK ✅ аномалий по высоте не найдено")
    else:
        print("WARN ⚠️ найдены аномалии по высоте:")
        print(df_ele)

    # 5) point_density sanity
    show("5) ПРОВЕРКА point_density_per_km")
    q_density_anoms = """
    SELECT f.track_id, f.distance_m, f.point_density_per_km, t.point_count
    FROM track_features f
    JOIN tracks t ON t.track_id = f.track_id
    WHERE
      (f.distance_m IS NOT NULL AND f.distance_m > 0 AND t.point_count > 0
        AND (f.point_density_per_km IS NULL OR f.point_density_per_km <= 0))
      OR (f.point_density_per_km IS NOT NULL AND f.point_density_per_km > 50000)
    ORDER BY f.point_density_per_km DESC NULLS LAST
    LIMIT 50;
    """
    df_den = pd.read_sql(text(q_density_anoms), conn)
    if df_den.empty:
        print("OK ✅ плотность точек выглядит нормально")
    else:
        print("WARN ⚠️ подозрительная плотность точек:")
        print(df_den)

    # 6) подозрительные длительности/скорости (time “странное”)
    show("6) ПОДОЗРИТЕЛЬНЫЕ ДЛИТЕЛЬНОСТИ/СКОРОСТИ (для треков с time)")
    q_time_weird = """
    SELECT
      f.track_id,
      t.track_name,
      t.start_time,
      t.end_time,
      f.distance_m,
      f.duration_s,
      f.avg_speed_mps,
      f.stop_time_s,
      f.stop_ratio
    FROM track_features f
    JOIN tracks t ON t.track_id = f.track_id
    WHERE
      (f.duration_s IS NOT NULL AND f.duration_s > 60*60*24*10)
      OR (f.avg_speed_mps IS NOT NULL AND (f.avg_speed_mps < 0 OR f.avg_speed_mps > 30))
      OR (f.stop_ratio IS NOT NULL AND (f.stop_ratio < 0 OR f.stop_ratio > 1.05))
    ORDER BY f.duration_s DESC NULLS LAST
    LIMIT 50;
    """
    df_weird = pd.read_sql(text(q_time_weird), conn)
    if df_weird.empty:
        print("OK ✅ экстремальных time-аномалий не найдено")
    else:
        print("WARN ⚠️ найдены подозрительные треки (скорее всего time в данных “не как трек”):")
        print(df_weird)

    # 7) примеры
    show("7) ПРИМЕРЫ track_features (первые 10 строк)")
    print(pd.read_sql(text("SELECT * FROM track_features ORDER BY track_id LIMIT 10;"), conn))

print("\n✅ CHECK features завершён")

OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public

0) ОБЪЁМ И СВЯЗНОСТЬ tracks / track_points / track_features
              tbl   rows
0  track_features     42
1    track_points  86457
2          tracks     42
   tracks_without_features
0                        0
   features_without_tracks
0                        0

1) ПОКРЫТИЕ И NULL-ПРОФИЛЬ ФИЧЕЙ (сколько треков с данными)
   tracks_total  with_distance  with_duration  with_avg_speed  with_max_speed  \
0            42             42              9               9               0   

   with_elev_min  with_elev_gain  with_stop_time  with_stop_ratio  
0             19              19               9                9  

2) СОГЛАСОВАННОСТЬ distance ↔ duration ↔ avg_speed
   track_id    distance_m   duration_s  avg_speed_mps  speed_calc_mps  \
0         2  1.114268e+04     83638.75       0.133224        0.133224   
1         3  1.034899e+02        59.00       1.754065        1.754065   
2         4  1.47363

In [63]:
import pandas as pd
from sqlalchemy import text
from src.common.db import get_engine, DBConfig, test_connection

test_connection(verbose=True)
engine = get_engine()
schema = DBConfig.from_env().schema

def show(title):
    print("\n" + "="*100)
    print(title)
    print("="*100)

# 1) Общая статистика
show("1) Общая статистика context_time_series")
q = f"""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT source_id) AS sources,
  MIN(time) AS min_time,
  MAX(time) AS max_time
FROM "{schema}"."context_time_series";
"""
print(pd.read_sql(text(q), engine))

# 2) Какие ключи внутри values
show("2) Ключи внутри values (JSON)")
q = f"""
SELECT key, COUNT(*) AS cnt
FROM "{schema}"."context_time_series",
LATERAL jsonb_object_keys(values) AS key
GROUP BY key
ORDER BY cnt DESC;
"""
print(pd.read_sql(text(q), engine))

# 3) Сколько записей с погодой / без погоды
show("3) Покрытие weather / nearby")
q = f"""
SELECT
  SUM(CASE WHEN values ? 'nearby' THEN 1 ELSE 0 END) AS with_nearby,
  SUM(CASE WHEN values ? 'weather' THEN 1 ELSE 0 END) AS with_weather,
  SUM(CASE WHEN (values->'weather'->>'ok') = 'true' THEN 1 ELSE 0 END) AS weather_ok,
  SUM(CASE WHEN (values->'weather'->>'ok') = 'false' THEN 1 ELSE 0 END) AS weather_fail
FROM "{schema}"."context_time_series";
"""
print(pd.read_sql(text(q), engine))

# 4) Показать последние 20 строк (чтобы было видно содержимое)
show("4) Последние 20 строк (коротко)")
q = f"""
SELECT
  context_id,
  source_id,
  time,
  lat,
  lon,
  values->'point'   AS point,
  values->'weather' AS weather,
  values->'nearby'  AS nearby
FROM "{schema}"."context_time_series"
ORDER BY context_id DESC
LIMIT 20;
"""
df_last = pd.read_sql(text(q), engine)
pd.set_option("display.max_colwidth", 200)
display(df_last)

# 5) Показать 10 строк в 'красивом' виде — отдельно температура и top теги nearby
show("5) Температура и топ-объекты рядом (10 строк)")
q = f"""
SELECT
  context_id,
  source_id,
  time,
  (values->'weather'->>'ok') AS weather_ok,
  (values->'weather'->>'temp_c') AS temp_c,
  values->'nearby'->'counts' AS nearby_counts
FROM "{schema}"."context_time_series"
ORDER BY context_id DESC
LIMIT 10;
"""
df_view = pd.read_sql(text(q), engine)
display(df_view)

OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public

1) Общая статистика context_time_series
   rows  sources                  min_time                         max_time
0    89        7 2010-01-01 00:43:23+00:00 2026-02-03 09:00:01.808081+00:00

2) Ключи внутри values (JSON)
        key  cnt
0  track_id   89
1    nearby   89
2     point   89
3   weather   89

3) Покрытие weather / nearby
   with_nearby  with_weather  weather_ok  weather_fail
0           89            89          84             5

4) Последние 20 строк (коротко)


,context_id,source_id,time,lat,lon,point,weather,nearby
0,89,8,2010-01-01 00:43:23+00:00,47.278490,-2.005200,"{'seq': 21000, 'ele_m': 4.765625, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 5.4, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'highway': [['unclassified', 5]], 'landuse': [['industrial', 1]], 'natural': [['water', 2], ['wood', 1]], 'building': [['yes', 10]]}, 'examples': [{'tag': 'highway', 'name': None, 'val..."
1,88,8,2010-01-01 00:43:23+00:00,47.202327,-1.678416,"{'seq': 20700, 'ele_m': 9.9140625, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 5.4, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['parking', 6], ['bicycle_parking', 1], ['recycling', 1], ['restaurant', 1], ['workshop', 1]], 'highway': [['service', 14], ['footway', 8], ['tertiary', 5], ['stop', 4], ['..."
2,87,8,2010-01-01 00:43:23+00:00,47.277662,-1.410643,"{'seq': 20400, 'ele_m': 7.2851563, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 5.3, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['bench', 1], ['waste_basket', 1]], 'highway': [['track', 1]], 'landuse': [['farmland', 1]], 'natural': [['water', 1]], 'waterway': [['river', 1]]}, 'examples': [{'tag': 'a..."
3,86,8,2010-01-01 00:43:23+00:00,47.364394,-1.070696,"{'seq': 19800, 'ele_m': 10.0, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 4.9, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'highway': [['service', 2], ['unclassified', 1]], 'landuse': [['meadow', 1], ['farmland', 1]], 'building': [['yes', 7]]}, 'examples': [{'tag': 'landuse', 'name': None, 'value': 'meadow..."
4,85,8,2010-01-01 00:43:23+00:00,47.361842,-0.753890,"{'seq': 19500, 'ele_m': 15.0, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 4.8, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'highway': [['unclassified', 2]], 'landuse': [['farmland', 1], ['forest', 1], ['meadow', 1]], 'natural': [['water', 1]], 'building': [['yes', 12]], 'waterway': [['river', 1]]}, 'exampl..."
5,84,8,2010-01-01 00:43:23+00:00,47.415088,-0.614666,"{'seq': 19200, 'ele_m': 18.1914063, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 4.7, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['bicycle_parking', 2], ['events_venue', 1], ['boat_rental', 1], ['recycling', 1], ['drinking_water', 1]], 'highway': [['service', 12], ['path', 7], ['secondary', 3], ['bus..."
6,83,8,2010-01-01 00:43:23+00:00,47.392697,-0.286220,"{'seq': 18900, 'ele_m': 33.7304688, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 4.6, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['watering_place', 1]], 'highway': [['tertiary', 3], ['track', 2], ['service', 1]], 'landuse': [['meadow', 1], ['vineyard', 1], ['residential', 1]], 'natural': [['water', 1..."
7,82,8,2010-01-01 00:43:23+00:00,47.217424,0.056004,"{'seq': 18300, 'ele_m': 32.2226563, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 5.0, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['bench', 13], ['bicycle_parking', 10], ['parking', 7], ['cafe', 4], ['waste_basket', 3]], 'highway': [['service', 19], ['path', 17], ['footway', 13], ['residential', 10], ..."
8,81,8,2010-01-01 00:43:23+00:00,47.205223,0.180568,"{'seq': 18000, 'ele_m': 34.1523438, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 5.0, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'amenity': [['bench', 6], ['parking', 5], ['waste_basket', 4], ['bicycle_parking', 4], ['cafe', 1]], 'highway': [['crossing', 15], ['service', 12], ['secondary', 5], ['cycleway', 5], [..."
9,80,8,2010-01-01 00:43:23+00:00,47.273131,0.320501,"{'seq': 17700, 'ele_m': 39.0, 'segment_index': 0}","{'ok': True, 'source': 'open-meteo-archive', 'temp_c': 4.6, 'hour_utc': '2010-01-01T00:00'}","{'counts': {'highway': [['tertiary', 3], ['unclassified', 3], ['service', 2], ['give_way', 1], ['tra


5) Температура и топ-объекты рядом (10 строк)


,context_id,source_id,time,weather_ok,temp_c,nearby_counts
0,89,8,2010-01-01 00:43:23+00:00,true,5.4,"{'highway': [['unclassified', 5]], 'landuse': [['industrial', 1]], 'natural': [['water', 2], ['wood', 1]], 'building': [['yes', 10]]}"
1,88,8,2010-01-01 00:43:23+00:00,true,5.4,"{'amenity': [['parking', 6], ['bicycle_parking', 1], ['recycling', 1], ['restaurant', 1], ['workshop', 1]], 'highway': [['service', 14], ['footway', 8], ['tertiary', 5], ['stop', 4], ['give_way', ..."
2,87,8,2010-01-01 00:43:23+00:00,true,5.3,"{'amenity': [['bench', 1], ['waste_basket', 1]], 'highway': [['track', 1]], 'landuse': [['farmland', 1]], 'natural': [['water', 1]], 'waterway': [['river', 1]]}"
3,86,8,2010-01-01 00:43:23+00:00,true,4.9,"{'highway': [['service', 2], ['unclassified', 1]], 'landuse': [['meadow', 1], ['farmland', 1]], 'building': [['yes', 7]]}"
4,85,8,2010-01-01 00:43:23+00:00,true,4.8,"{'highway': [['unclassified', 2]], 'landuse': [['farmland', 1], ['forest', 1], ['meadow', 1]], 'natural': [['water', 1]], 'building': [['yes', 12]], 'waterway': [['river', 1]]}"
5,84,8,2010-01-01 00:43:23+00:00,true,4.7,"{'amenity': [['bicycle_parking', 2], ['events_venue', 1], ['boat_rental', 1], ['recycling', 1], ['drinking_water', 1]], 'highway': [['service', 12], ['path', 7], ['secondary', 3], ['bus_stop', 2],..."
6,83,8,2010-01-01 00:43:23+00:00,true,4.6,"{'amenity': [['watering_place', 1]], 'highway': [['tertiary', 3], ['track', 2], ['service', 1]], 'landuse': [['meadow', 1], ['vineyard', 1], ['residential', 1]], 'natural': [['water', 1]], 'buildi..."
7,82,8,2010-01-01 00:43:23+00:00,true,5.0,"{'amenity': [['bench', 13], ['bicycle_parking', 10], ['parking', 7], ['cafe', 4], ['waste_basket', 3]], 'highway': [['service', 19], ['path', 17], ['footway', 13], ['residential', 10], ['crossing'..."
8,81,8,2010-01-01 00:43:23+00:00,true,5.0,"{'amenity': [['bench', 6], ['parking', 5], ['waste_basket', 4], ['bicycle_parking', 4], ['cafe', 1]], 'highway': [['crossing', 15], ['service', 12], ['secondary', 5], ['cycleway', 5], ['path', 4]]..."
9,80,8,2010-01-01 00:43:23+00:00,true,4.6,"{'highway': [['tertiary', 3], ['unclassified', 3], ['service', 2], ['give_way', 1], ['track', 1]], 'landuse': [['residential', 1]], 'natural': [['water', 1]], 'building': [['yes', 49]], 'waterway'..."


In [45]:
import pandas as pd
from sqlalchemy import text
from src.common.db import get_engine, DBConfig

engine = get_engine()
schema = DBConfig.from_env().schema

df = pd.read_sql(text(f'SELECT * FROM "{schema}"."context_time_series" ORDER BY context_id'), engine)
df.to_csv("context_time_series_dump.csv", index=False)
print("Сохранено: context_time_series_dump.csv | строк:", len(df))

Сохранено: context_time_series_dump.csv | строк: 15


In [46]:
from sqlalchemy import text
from src.common.db import get_engine, test_connection

# 1) Проверяем подключение (на всякий случай)
test_connection(verbose=True)

engine = get_engine()

# 2) Полная очистка таблицы context_time_series
with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE context_time_series RESTART IDENTITY;"))

print("✅ Таблица context_time_series полностью очищена")

OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public
✅ Таблица context_time_series полностью очищена


In [64]:
import json
from sqlalchemy import text
from pprint import pprint

from src.common.db import get_engine, set_search_path

engine = get_engine()

with engine.connect() as conn:
    set_search_path(conn)

    # Берём 1 любую строку, где есть weather
    row = conn.execute(text("""
        SELECT context_id, values
        FROM context_time_series
        WHERE values ? 'weather'
        LIMIT 1
    """)).fetchone()

context_id, values = row

print(f"\n📦 context_id = {context_id}")
print("\n🔎 ПОЛНОЕ СОДЕРЖИМОЕ JSON values:\n")

# values уже dict (SQLAlchemy сам распарсил jsonb)
pprint(values, width=120)


📦 context_id = 1

🔎 ПОЛНОЕ СОДЕРЖИМОЕ JSON values:

{'nearby': {'counts': {'amenity': [['bench', 10],
                                   ['parking', 4],
                                   ['bicycle_parking', 2],
                                   ['parking_entrance', 2],
                                   ['waste_basket', 1]],
                       'building': [['apartments', 39],
                                    ['detached', 13],
                                    ['school', 11],
                                    ['container', 5],
                                    ['construction', 5]],
                       'highway': [['footway', 120],
                                   ['service', 16],
                                   ['residential', 11],
                                   ['steps', 6],
                                   ['street_lamp', 4]],
                       'landuse': [['residential', 6],
                                   ['grass', 4],
                          